<a href="https://colab.research.google.com/github/RAHEYO/SQuAD-MRC/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation


In [ ]:
# prompt: install tensorflow, pennylane, and all the related libraries that may be needed to use those two libraries.

!pip install autograd tensorflow jax jaxlib
!pip install pennylane
!pip install pennylane-lightning pennylane-lightning[gpu]
!pip install numpy



In [ ]:
# prompt: import tensorflow and pennylane

import tensorflow as tf
import pennylane as qml
import numpy as np


# Data Prep

In [ ]:

dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def circuit(params):
    # ...
